# model

In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define data directories
train_dir = 'data3/' 
test_dir = 'data/test/'

# Image dimensions
img_width, img_height = 150, 150

# Training parameters
batch_size = 32
epochs = 10

# Data generators with data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

# Build the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

# Evaluate the model
score = model.evaluate(test_generator)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])

model.save('spoof2.h5')


2023-07-20 21:10:32.827748: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-20 21:10:37.235749: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-20 21:10:37.275585: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-20 21:10:46.507429: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 6164 images belonging to 2 classes.
Found 951 images belonging to 2 classes.


2023-07-20 21:11:06.737149: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.
2023-07-20 21:11:06.797488: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.
2023-07-20 21:11:06.897761: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.


Epoch 1/10


2023-07-20 21:11:09.360600: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.
2023-07-20 21:11:09.367985: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.


192/192 [==============================] - 478s 2s/step - loss: 0.5422 - accuracy: 0.7208 - val_loss: 0.3241 - val_accuracy: 0.8534
Epoch 2/10
192/192 [==============================] - 271s 1s/step - loss: 0.4423 - accuracy: 0.7916 - val_loss: 0.3876 - val_accuracy: 0.8082
Epoch 3/10
192/192 [==============================] - 247s 1s/step - loss: 0.4300 - accuracy: 0.7988 - val_loss: 0.3139 - val_accuracy: 0.8825
Epoch 4/10
192/192 [==============================] - 248s 1s/step - loss: 0.3831 - accuracy: 0.8144 - val_loss: 0.2261 - val_accuracy: 0.9116
Epoch 5/10
192/192 [==============================] - 258s 1s/step - loss: 0.3669 - accuracy: 0.8286 - val_loss: 0.1758 - val_accuracy: 0.9397
Epoch 6/10
192/192 [==============================] - 275s 1s/step - loss: 0.3464 - accuracy: 0.8434 - val_loss: 0.1837 - val_accuracy: 0.9332
Epoch 7/10
192/192 [==============================] - 256s 1s/step - loss: 0.3207 - accuracy: 0.8594 - val_loss: 0.1276 - val_accuracy: 0.9569
Epoch 8/10

# mediapipe

In [4]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

# Load the anti-spoofing model
anti_spoof_model = load_model('spoof2.h5')  # Replace with the path to your trained model

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils



# For webcam input:
cap = cv2.VideoCapture(0)
with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image_rgb)

        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = image.shape
                bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                       int(bboxC.width * iw), int(bboxC.height * ih)

                face_img = image[bbox[1]:bbox[1] + bbox[3], bbox[0]:bbox[0] + bbox[2]]
                face_img = cv2.resize(face_img, (150,150))
                face_img = np.expand_dims(face_img, axis=0)
                face_img = face_img / 255.0

                prediction = anti_spoof_model.predict(face_img)
                if prediction[0][0] < 0.5:
                    label = "Real"
                else:
                    label = "Spoof"

                mp_drawing.draw_detection(image, detection)
                cv2.putText(image, label, (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        cv2.imshow('MediaPipe Face Detection', image)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()


[ WARN:0@206.749] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@206.750] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range
